In [2]:
import pandas as pd
import csv
import numpy as np

In [3]:
train = 'train.txt'
labels = 'train_labels.txt'

In [4]:
train_df = pd.read_csv(train, sep=" ", engine="c", header=None)

In [5]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9
0,120.3882,67.1368,3.6905,0.1128,0.0481,-155.5746,35.3004,47.8095,3.8760,289.0430
1,28.5322,15.1782,2.6686,0.4306,0.2399,30.9627,21.7936,21.9248,62.7504,254.0526
2,38.8799,21.5881,3.0573,0.2025,0.1047,18.7566,-24.0106,11.6233,0.7040,185.2510
3,28.8255,16.7964,2.6642,0.3402,0.1701,3.6149,-18.2080,9.8634,2.4949,213.1690
4,20.7522,8.9492,2.3874,0.6434,0.3586,24.0649,10.5927,1.8768,36.3060,126.3930


In [18]:
train_df.shape

(15000, 10)

In [6]:
labels_df = pd.Series(pd.read_csv(labels, sep=" ", engine="c", header=None)[0])

In [7]:
labels_df.head()

0    1
1    1
2    0
3    0
4    1
Name: 0, dtype: int64

Normalize all columns to equalize the feature space vectors:

In [8]:
from sklearn import preprocessing
train_df = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(train_df))

In [9]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9
0,0.346150,0.261862,0.516422,0.113107,0.070825,0.287144,0.643887,0.673694,0.043067,0.582183
1,0.065218,0.059202,0.213772,0.474366,0.355016,0.469097,0.620195,0.604959,0.697227,0.511392
2,0.096865,0.084203,0.328891,0.215073,0.154690,0.457191,0.539851,0.577604,0.007822,0.372196
3,0.066115,0.065513,0.212469,0.371604,0.251593,0.442421,0.550029,0.572931,0.027721,0.428678
4,0.041424,0.034906,0.130490,0.716267,0.530893,0.462369,0.600547,0.551723,0.403400,0.253117


1. KNN
2. SVM
3. Random Forest
4. Gradient Boosting Trees

In [16]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [21]:
from sklearn.neighbors import KNeighborsClassifier

knn_neighbors = []
for i in range(9):
    knn_neighbors.append(i+1)

knn_cv_params = {
    'n_neighbors': knn_neighbors,
#     'algorithm': ['ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2, 3],
    'weights': ['uniform', 'distance']
}

knn_grid = GridSearchCV(KNeighborsClassifier(), param_grid=knn_cv_params)
knn_grid.fit(train_df, labels_df)
print(knn_grid.best_score_)
print(knn_grid.best_params_)

0.8392
{'weights': 'distance', 'p': 1, 'n_neighbors': 7}


In [54]:
from sklearn.svm import SVC

svm_cv_params = {
#     'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'probability': [True, False]
#     'C': [0.1, 0.5, 1.0]
}

svm_grid = GridSearchCV(SVC(cache_size=512, C=15.0, kernel='rbf'), param_grid=svm_cv_params)
svm_grid.fit(train_df, labels_df)
print(svm_grid.best_score_)
print(svm_grid.best_params_)

0.8408
{'class_weight': None, 'probability': True}


In [60]:
from sklearn.ensemble import RandomForestClassifier

rf_cv_params = {
    'n_estimators': [100, 300, 500],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2', None]
}

rf_grid = GridSearchCV(RandomForestClassifier(), param_grid=rf_cv_params)
rf_grid.fit(train_df, labels_df)
print(rf_grid.best_score_)
print(rf_grid.best_params_)

0.8767333333333334
{'max_features': 'log2', 'criterion': 'gini', 'n_estimators': 100}


In [64]:
from sklearn.ensemble import GradientBoostingClassifier

gbt_cv_params = {
    'loss': ['deviance', 'exponential'],
#     'n_estimators': [100, 300, 500, 700],
    'n_estimators': [700, 900, 1100],
    'max_features': ['sqrt', 'log2', None]
}

gbt_grid = GridSearchCV(GradientBoostingClassifier(), param_grid=gbt_cv_params)
gbt_grid.fit(train_df, labels_df)
print(gbt_grid.best_score_)
print(gbt_grid.best_params_)

0.882
{'loss': 'deviance', 'max_features': 'sqrt', 'n_estimators': 1100}
